# Import module 

Most codes from https://github.com/hwalsuklee/tensorflow-generative-model-collections/blob/master/GAN.py

In [1]:
import os
import scipy
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Loading MNIST data

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='2'
mnist = input_data.read_data_sets("data/mnist")

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


# Discriminator Network

For easy reading，I did not put the "get_variable","conv2d" into one function

In [3]:
def discriminator(inputs, batch_size, is_training=True, reuse=False):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : (64)4c2s-(128)4c2s_BL-FC1024_BL-FC1_S
    
    with tf.variable_scope("discriminator", reuse=reuse):
        
        '''1st: Conv -> lrelu'''
        #conv
        w1 = tf.get_variable('d_wconv1', [4, 4, 1, 64],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b1 = tf.get_variable('d_bconv1', [64], initializer=tf.constant_initializer(0.0))
        net = tf.nn.conv2d(inputs, w1, strides=[1, 2, 2, 1], padding='SAME')
        net = tf.reshape(tf.nn.bias_add(net, b1), net.get_shape())
        #lrelu
        net = tf.maximum(net, 0.2*net)
        
        '''2nd: Conv -> bn -> lrelu'''
        #conv
        w2 = tf.get_variable('d_wconv2', [4, 4, 64, 128],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b2 = tf.get_variable('d_bconv2', [128], initializer=tf.constant_initializer(0.0))
        net = tf.nn.conv2d(net, w2, strides=[1, 2, 2, 1], padding='SAME')
        #bn
        net = tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                            is_training=is_training, scope='d_bn2')
        net = tf.reshape(tf.nn.bias_add(net, b2), net.get_shape())
        #lrelu
        net = tf.maximum(net, 0.2*net)
        
        net = tf.reshape(net, [batch_size, -1])
        
        '''3th: linear -> bn -> lrelu'''
        #linear
        shape = net.get_shape().as_list()        
        w3 = tf.get_variable("d_wlinear3", [shape[1], 1024], tf.float32,tf.random_normal_initializer(stddev=0.02))
        b3 = tf.get_variable("d_blinear3", [1024], initializer=tf.constant_initializer(0.0))
        net = tf.matmul(net, w3) + b3
        #bn
        net = tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                        is_training=is_training, scope='d_bn3')
        #lrelu
        net = tf.maximum(net, 0.2*net)
        
        '''4th: linear '''
        #linear
        shape = net.get_shape().as_list()        
        w4 = tf.get_variable("d_wlinear4", [shape[1], 1], tf.float32,tf.random_normal_initializer(stddev=0.02))
        b4 = tf.get_variable("d_blinear4", [1], initializer=tf.constant_initializer(0.0))
        out_logit = tf.matmul(net, w4) + b4
        
        '''5th: sigmoid'''
        out = tf.nn.sigmoid(out_logit)        
        
        return out, out_logit


# Generator Network

In [5]:
def generator( z,batch_size, is_training=True, reuse=False):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S
    
    with tf.variable_scope("generator", reuse=reuse):
        
        '''1st: linear -> bn -> relu '''
        #linear
        shape = z.get_shape().as_list()        
        w1 = tf.get_variable("g_wlinear1", [shape[1], 1024], tf.float32,tf.random_normal_initializer(stddev=0.02))
        b1 = tf.get_variable("g_blinear1", [1024], initializer=tf.constant_initializer(0.0))
        net = tf.matmul(z, w1) + b1
        #bn
        net = tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                        is_training=is_training, scope='g_bn1')
        #relu
        net = tf.nn.relu(net)  
        
        '''2nd: linear -> bn -> relu '''
        #linear
        shape = net.get_shape().as_list()        
        w2 = tf.get_variable("g_wlinear2", [shape[1], 128*7*7], tf.float32,tf.random_normal_initializer(stddev=0.02))
        b2 = tf.get_variable("g_blinear2", [128*7*7], initializer=tf.constant_initializer(0.0))
        net = tf.matmul(net, w2) + b2
        #bn
        net = tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                        is_training=is_training, scope='g_bn2')
        #relu
        net = tf.nn.relu(net) 
                     
        net = tf.reshape(net, [batch_size, 7, 7, 128])   
        
        '''3th: deconv -> bn -> relu '''
        #deconv
        output_shape = [batch_size, 14, 14, 64]
        w3 = tf.get_variable('g_wdeconv3', [4, 4, output_shape[-1], net.get_shape()[-1]],
                                           initializer=tf.random_normal_initializer(stddev=0.02))
        net = tf.nn.conv2d_transpose(net, w3, output_shape=output_shape, strides=[1, 2, 2, 1])
        b3 = tf.get_variable('g_bdeconv3', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        net = tf.reshape(tf.nn.bias_add(net, b3), net.get_shape())
        #bn
        net = tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                        is_training=is_training, scope='g_bn3')
        #relu
        net = tf.nn.relu(net)
        
        '''4th: deconv -> bn -> sigmoid '''
        #deconv
        output_shape = [batch_size, 28, 28, 1]
        w4 = tf.get_variable('g_wdeconv4', [4, 4, output_shape[-1], net.get_shape()[-1]],
                                           initializer=tf.random_normal_initializer(stddev=0.02))
        net = tf.nn.conv2d_transpose(net, w4, output_shape=output_shape, strides=[1, 2, 2, 1])
        b4 = tf.get_variable('g_bdeconv4', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        net = tf.reshape(tf.nn.bias_add(net, b4), net.get_shape())
        #sigmoid
        out = tf.nn.sigmoid(net)

        return out


# set the global parameters.

In [6]:
# some parameters
image_dims = [28, 28, 1]
batch_size = 64
z_dim = 62
learning_rate = 0.0002
beta1 = 0.5

""" Graph Input """
# images
inputs = tf.placeholder(tf.float32, [batch_size] + image_dims, name='real_images')
# noises
z = tf.placeholder(tf.float32, [batch_size, z_dim], name='z')

# Loss Function 

In [7]:
""" Loss Function """

# output of D for real images
D_real, D_real_logits = discriminator(inputs, batch_size, is_training=True, reuse=False)

# output of D for fake images
G = generator(z, batch_size, is_training=True, reuse=False)
D_fake, D_fake_logits = discriminator(G, batch_size, is_training=True, reuse=True)

# get loss for discriminator
d_loss_real = tf.reduce_mean(
              tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real)))
d_loss_fake = tf.reduce_mean(
              tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake)))
d_loss = d_loss_real + d_loss_fake

# get loss for generator
g_loss = tf.reduce_mean(
         tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake)))


split the generator parameters and discriminator parameters into two list, then define how to train the two subnetwork and get the fake image for testing.

In [8]:
""" Training """
# divide trainable variables into a group for D and a group for G
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'd_' in var.name]
g_vars = [var for var in t_vars if 'g_' in var.name]
# optimizers
d_optim = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(d_loss, var_list=d_vars)
g_optim = tf.train.AdamOptimizer(learning_rate*5, beta1=beta1).minimize(g_loss, var_list=g_vars)


"""" Testing """
# for test
fake_images = generator(z,batch_size, is_training=False, reuse=True)
# graph inputs for visualize training results
sample_z = np.random.uniform(-1, 1, size=(batch_size , z_dim))

# Training a GAN

In [ ]:
start_epoch = 0
start_batch_id = 0


num_steps = 6000
# loop for epoch
start_time = time.time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step_ind in range(num_steps):
    
    '''get the real data'''
    real_image_batch = mnist.train.next_batch(batch_size)
    batch_images = np.reshape(real_image_batch[0],[batch_size,28,28,1]).astype(np.float32)
    '''get the noise data'''
    batch_z = np.random.uniform(-1, 1, [batch_size, z_dim]).astype(np.float32)

    # update D network
    _ , D_loss = sess.run([d_optim, d_loss], feed_dict={inputs: batch_images, z: batch_z})
    # update G network
    _, G_loss = sess.run([g_optim,g_loss], feed_dict={z: batch_z})

    # display training status
    print("Step: [%d] d_loss: %.8f, g_loss: %.8f" % (step_ind, D_loss, G_loss) )

    # save training results for every 300 steps
    if np.mod(counter, 300) == 0:

        samples = sess.run(fake_images, feed_dict={z: sample_z})
        # put the "batch_size" images into one big canvas
        row = col = int(np.sqrt(batch_size))
        img = np.zeros( [row*28, col*28] )
        for i in range(row):
            for j in range(col):
                img[i*28:(i+1)*28,j*28:(j+1)*28] = samples[i*col+j, :, :, :].squeeze()
        #save the result      
        scipy.misc.imsave('{}.jpg'.format(step_ind),img)

Step: [0] d_loss: 1.46458840, g_loss: 0.64777845
Step: [1] d_loss: 1.46579719, g_loss: 0.65457553
Step: [2] d_loss: 1.42761779, g_loss: 0.67831904
Step: [3] d_loss: 1.42042303, g_loss: 0.69852638
Step: [4] d_loss: 1.40911579, g_loss: 0.71736163
Step: [5] d_loss: 1.39809585, g_loss: 0.73251450
Step: [6] d_loss: 1.39723253, g_loss: 0.74750620
Step: [7] d_loss: 1.38970423, g_loss: 0.75682068
Step: [8] d_loss: 1.37036037, g_loss: 0.76273489
Step: [9] d_loss: 1.38706350, g_loss: 0.75738072
Step: [10] d_loss: 1.36794412, g_loss: 0.76199967
Step: [11] d_loss: 1.37723041, g_loss: 0.76501459
Step: [12] d_loss: 1.36540079, g_loss: 0.76265597
Step: [13] d_loss: 1.34757054, g_loss: 0.77868283
Step: [14] d_loss: 1.34565091, g_loss: 0.77489394
Step: [15] d_loss: 1.36014152, g_loss: 0.77082920
Step: [16] d_loss: 1.33849001, g_loss: 0.78033423
Step: [17] d_loss: 1.34064722, g_loss: 0.78339583
Step: [18] d_loss: 1.35505939, g_loss: 0.77385914
Step: [19] d_loss: 1.34183013, g_loss: 0.78090453
Step: [20]

Step: [163] d_loss: 1.16021383, g_loss: 0.90096569
Step: [164] d_loss: 1.13746166, g_loss: 0.93918234
Step: [165] d_loss: 1.29154968, g_loss: 0.92979228
Step: [166] d_loss: 1.19562268, g_loss: 0.92627609
Step: [167] d_loss: 1.25973105, g_loss: 0.90020436
Step: [168] d_loss: 1.13062263, g_loss: 0.90589327
Step: [169] d_loss: 1.11436629, g_loss: 0.92416775
Step: [170] d_loss: 1.17126727, g_loss: 0.86015511
Step: [171] d_loss: 1.18663919, g_loss: 0.90839958
Step: [172] d_loss: 1.19090140, g_loss: 0.92560428
Step: [173] d_loss: 1.18500829, g_loss: 0.93888623
Step: [174] d_loss: 1.29322529, g_loss: 0.91154134
Step: [175] d_loss: 1.21504068, g_loss: 0.99424374
Step: [176] d_loss: 1.32586646, g_loss: 0.95394939
Step: [177] d_loss: 1.28439057, g_loss: 0.90463310
Step: [178] d_loss: 1.26655209, g_loss: 0.86106753
Step: [179] d_loss: 1.24316525, g_loss: 0.86450154
Step: [180] d_loss: 1.23305821, g_loss: 0.91394925
Step: [181] d_loss: 1.19141841, g_loss: 0.89416999
Step: [182] d_loss: 1.25196147,

Step: [327] d_loss: 1.24658334, g_loss: 0.84208733
Step: [328] d_loss: 1.32064581, g_loss: 0.80755973
Step: [329] d_loss: 1.26866007, g_loss: 0.83582515
Step: [330] d_loss: 1.27691209, g_loss: 0.82558954
Step: [331] d_loss: 1.34289765, g_loss: 0.84369421
Step: [332] d_loss: 1.27578294, g_loss: 0.84940726
Step: [333] d_loss: 1.28198922, g_loss: 0.84183192
Step: [334] d_loss: 1.29296589, g_loss: 0.83730906
Step: [335] d_loss: 1.27596307, g_loss: 0.84055465
Step: [336] d_loss: 1.30536985, g_loss: 0.82763636
Step: [337] d_loss: 1.31339622, g_loss: 0.81619519
Step: [338] d_loss: 1.29856896, g_loss: 0.86170655
Step: [339] d_loss: 1.30065000, g_loss: 0.80911672
Step: [340] d_loss: 1.28131664, g_loss: 0.78773981
Step: [341] d_loss: 1.24631858, g_loss: 0.80717397
Step: [342] d_loss: 1.29670191, g_loss: 0.80046040
Step: [343] d_loss: 1.28210974, g_loss: 0.85831815
Step: [344] d_loss: 1.26836264, g_loss: 0.85105300
Step: [345] d_loss: 1.29909682, g_loss: 0.85288274
Step: [346] d_loss: 1.33700538,

Step: [492] d_loss: 1.31919992, g_loss: 0.82600820
Step: [493] d_loss: 1.29894137, g_loss: 0.82632840
Step: [494] d_loss: 1.32972932, g_loss: 0.80943882
Step: [495] d_loss: 1.29131925, g_loss: 0.80868787
Step: [496] d_loss: 1.32562935, g_loss: 0.80618685
Step: [497] d_loss: 1.29935098, g_loss: 0.80482489
Step: [498] d_loss: 1.28808022, g_loss: 0.83069849
Step: [499] d_loss: 1.27031851, g_loss: 0.82768548
Step: [500] d_loss: 1.32106447, g_loss: 0.81960690
Step: [501] d_loss: 1.31202292, g_loss: 0.79664785
Step: [502] d_loss: 1.32104421, g_loss: 0.82707864
Step: [503] d_loss: 1.28995538, g_loss: 0.83127499
Step: [504] d_loss: 1.28819084, g_loss: 0.80740374
Step: [505] d_loss: 1.28957784, g_loss: 0.81623751
Step: [506] d_loss: 1.31816471, g_loss: 0.79930735
Step: [507] d_loss: 1.27488804, g_loss: 0.82572919
Step: [508] d_loss: 1.31626678, g_loss: 0.81521809
Step: [509] d_loss: 1.31395650, g_loss: 0.79788440
Step: [510] d_loss: 1.29861200, g_loss: 0.81360614
Step: [511] d_loss: 1.30751872,

Step: [657] d_loss: 1.36312556, g_loss: 0.77613235
Step: [658] d_loss: 1.29463995, g_loss: 0.82174325
Step: [659] d_loss: 1.32012284, g_loss: 0.81341636
Step: [660] d_loss: 1.33217204, g_loss: 0.80624276
Step: [661] d_loss: 1.32786918, g_loss: 0.80297601
Step: [662] d_loss: 1.29863572, g_loss: 0.82023185
Step: [663] d_loss: 1.27490652, g_loss: 0.82199132
Step: [664] d_loss: 1.33112705, g_loss: 0.82189906
Step: [665] d_loss: 1.32108426, g_loss: 0.79312301
Step: [666] d_loss: 1.31127477, g_loss: 0.80638045
Step: [667] d_loss: 1.31000996, g_loss: 0.80903995
Step: [668] d_loss: 1.30697501, g_loss: 0.81195432
Step: [669] d_loss: 1.36269605, g_loss: 0.79791838
Step: [670] d_loss: 1.30798066, g_loss: 0.80204159
Step: [671] d_loss: 1.31512094, g_loss: 0.80292547
Step: [672] d_loss: 1.35746884, g_loss: 0.79257369
Step: [673] d_loss: 1.34614563, g_loss: 0.80340099
Step: [674] d_loss: 1.31588888, g_loss: 0.80986667
Step: [675] d_loss: 1.32730138, g_loss: 0.80052900
Step: [676] d_loss: 1.28981042,

...

Step: [3062] d_loss: 1.29957914, g_loss: 0.84229624
Step: [3063] d_loss: 1.30592608, g_loss: 0.84846067
Step: [3064] d_loss: 1.30920970, g_loss: 0.90818465
Step: [3065] d_loss: 1.30763626, g_loss: 0.85381472
Step: [3066] d_loss: 1.32627892, g_loss: 0.86748570
Step: [3067] d_loss: 1.27348161, g_loss: 0.87119138
Step: [3068] d_loss: 1.28569090, g_loss: 0.85765779
Step: [3069] d_loss: 1.29495597, g_loss: 0.85902846
Step: [3070] d_loss: 1.25057173, g_loss: 0.86141729
Step: [3071] d_loss: 1.29803038, g_loss: 0.83419067
Step: [3072] d_loss: 1.24335957, g_loss: 0.87135625
Step: [3073] d_loss: 1.26991081, g_loss: 0.85273230
Step: [3074] d_loss: 1.26514685, g_loss: 0.86853087
Step: [3075] d_loss: 1.29536986, g_loss: 0.85885322
Step: [3076] d_loss: 1.30284989, g_loss: 0.86306632
Step: [3077] d_loss: 1.28280425, g_loss: 0.87169135
Step: [3078] d_loss: 1.27624428, g_loss: 0.83554369
Step: [3079] d_loss: 1.31083679, g_loss: 0.83037835
Step: [3080] d_loss: 1.23816919, g_loss: 0.87953651
Step: [3081]

Step: [3222] d_loss: 1.25809407, g_loss: 0.88059151
Step: [3223] d_loss: 1.26488662, g_loss: 0.89735699
Step: [3224] d_loss: 1.27736855, g_loss: 0.87499654
Step: [3225] d_loss: 1.31456065, g_loss: 0.87054443
Step: [3226] d_loss: 1.25037694, g_loss: 0.87725341
Step: [3227] d_loss: 1.28260124, g_loss: 0.85831034
Step: [3228] d_loss: 1.27809024, g_loss: 0.86056262
Step: [3229] d_loss: 1.26027799, g_loss: 0.88681269
Step: [3230] d_loss: 1.31218755, g_loss: 0.90058547
Step: [3231] d_loss: 1.39055133, g_loss: 0.85053122
Step: [3232] d_loss: 1.33778167, g_loss: 0.80553216
Step: [3233] d_loss: 1.27211785, g_loss: 0.82776582
Step: [3234] d_loss: 1.31666648, g_loss: 0.83567870
Step: [3235] d_loss: 1.31970465, g_loss: 0.85253465
Step: [3236] d_loss: 1.32733870, g_loss: 0.85349530
Step: [3237] d_loss: 1.27098608, g_loss: 0.87636149
Step: [3238] d_loss: 1.34733367, g_loss: 0.82990050
Step: [3239] d_loss: 1.21864605, g_loss: 0.91020495
Step: [3240] d_loss: 1.31444812, g_loss: 0.84479213
Step: [3241]

Step: [3382] d_loss: 1.28218174, g_loss: 0.88974482
Step: [3383] d_loss: 1.27877939, g_loss: 0.84062493
Step: [3384] d_loss: 1.27514434, g_loss: 0.85384494
Step: [3385] d_loss: 1.30023360, g_loss: 0.85297376
Step: [3386] d_loss: 1.22108102, g_loss: 0.90575063
Step: [3387] d_loss: 1.30016649, g_loss: 0.86979723
Step: [3388] d_loss: 1.25325346, g_loss: 0.89607680
Step: [3389] d_loss: 1.30867600, g_loss: 0.85087639
Step: [3390] d_loss: 1.24445879, g_loss: 0.86207044
Step: [3391] d_loss: 1.30845475, g_loss: 0.89245522
Step: [3392] d_loss: 1.29488301, g_loss: 0.87490427
Step: [3393] d_loss: 1.25496483, g_loss: 0.88500059
Step: [3394] d_loss: 1.24542952, g_loss: 0.92096001
Step: [3395] d_loss: 1.28516698, g_loss: 0.85670125
Step: [3396] d_loss: 1.25821364, g_loss: 0.85652441
Step: [3397] d_loss: 1.24202073, g_loss: 0.86224920
Step: [3398] d_loss: 1.22504568, g_loss: 0.88445830
Step: [3399] d_loss: 1.26336420, g_loss: 0.85370231
Step: [3400] d_loss: 1.28869796, g_loss: 0.86520422
Step: [3401]

Step: [3541] d_loss: 1.24536407, g_loss: 0.85851973
Step: [3542] d_loss: 1.27318251, g_loss: 0.85040766
Step: [3543] d_loss: 1.25120211, g_loss: 0.90428698
Step: [3544] d_loss: 1.26428485, g_loss: 0.89775777
Step: [3545] d_loss: 1.24066794, g_loss: 0.90226603
Step: [3546] d_loss: 1.26421762, g_loss: 0.87585318
Step: [3547] d_loss: 1.26481640, g_loss: 0.85540485
Step: [3548] d_loss: 1.27002966, g_loss: 0.88370484
Step: [3549] d_loss: 1.22547162, g_loss: 0.88648129
Step: [3550] d_loss: 1.24505329, g_loss: 0.87473989
Step: [3551] d_loss: 1.25556374, g_loss: 0.85817492
Step: [3552] d_loss: 1.24588633, g_loss: 0.86195642
Step: [3553] d_loss: 1.22803891, g_loss: 0.87444311
Step: [3554] d_loss: 1.23996282, g_loss: 0.91698158
Step: [3555] d_loss: 1.26958334, g_loss: 0.91021454
Step: [3556] d_loss: 1.27239347, g_loss: 0.89522207
Step: [3557] d_loss: 1.28129363, g_loss: 0.83805406
Step: [3558] d_loss: 1.24280047, g_loss: 0.89178145
Step: [3559] d_loss: 1.20851755, g_loss: 0.89239061
Step: [3560]

Step: [3701] d_loss: 1.22062373, g_loss: 0.91746056
Step: [3702] d_loss: 1.20167017, g_loss: 0.87389481
Step: [3703] d_loss: 1.26975179, g_loss: 0.86403632
Step: [3704] d_loss: 1.26171756, g_loss: 0.87961423
Step: [3705] d_loss: 1.25960183, g_loss: 0.91804236
Step: [3706] d_loss: 1.30984139, g_loss: 0.84245509
Step: [3707] d_loss: 1.28483701, g_loss: 0.84993529
Step: [3708] d_loss: 1.25754714, g_loss: 0.87152189
Step: [3709] d_loss: 1.28665590, g_loss: 0.86150306
Step: [3710] d_loss: 1.28079402, g_loss: 0.87708133
